In [ ]:
import os
import shutil
import json

# Load task paths from tasks_path.json
tasks_path_file = "../example_code/tasks_path.json"
with open(tasks_path_file, "r") as f:
    tasks_path = json.load(f)

# Define the root path and source folder
root_path = "../storage_server/COLM_res_update/"
source_folder = "../logging/close_source/"

# Define the target folders
python_completion_folder = os.path.join(root_path, "Python_all_res/Completion")
python_infilling_folder = os.path.join(root_path, "Python_all_res/Infilling")
java_completion_folder = os.path.join(root_path, "Java_all_res/Completion")
java_infilling_folder = os.path.join(root_path, "Java_all_res/Infilling")

# Ensure target folders exist
os.makedirs(python_completion_folder, exist_ok=True)
os.makedirs(python_infilling_folder, exist_ok=True)
os.makedirs(java_completion_folder, exist_ok=True)
os.makedirs(java_infilling_folder, exist_ok=True)

# Flatten task paths for easy lookup
python_tasks = set(tasks_path["Python"].keys())
java_tasks = set(tasks_path["Java"].keys())

# Iterate through folders and files in the logging directory
for folder_name in os.listdir(source_folder):
    folder_path = os.path.join(source_folder, folder_name)
    if not os.path.isdir(folder_path):
        continue

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            source_file = os.path.join(folder_path, file_name)

            # Determine task type (Python or Java) based on file name
            task_type = None
            for task in python_tasks:
                if task in file_name:
                    task_type = "Python"
                    break
            for task in java_tasks:
                if task in file_name:
                    task_type = "Java"
                    break

            if not task_type:
                print(f"Skipping file (no matching task): {file_name}")
                continue

            # Determine target folder based on keywords in file name
            if "no_afterlines" in file_name:
                if task_type == "Python":
                    target_file = os.path.join(python_completion_folder, file_name)
                else:
                    target_file = os.path.join(java_completion_folder, file_name)
            elif "with_afterlines" in file_name:
                if task_type == "Python":
                    target_file = os.path.join(python_infilling_folder, file_name)
                else:
                    target_file = os.path.join(java_infilling_folder, file_name)
            else:
                print(f"Skipping file (no matching keyword): {file_name}")
                continue

            # Copy the file to the target folder
            shutil.copy(source_file, target_file)
            print(f"Copied: {file_name} --> {target_file}")